### Collection 객체 생성과 문서 저장

In [1]:
import chromadb

# collection을 저장할 경로 지정
client = chromadb.PersistentClient(path = "collection_example")

# client가 잘 연결되어 있는지 확인
client.heartbeat()

1743428438502729000

#### 1. Collection 생성

In [ ]:
from chromadb.utils import embedding_functions

# Huggingface 오픈소스 임베딩 모델 활용
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="jhgan/ko-sbert-nli")

# collection 생성
collection = client.create_collection(name="korean_law", embedding_function=embedding_function)

/opt/anaconda3/envs/RAG_CODE/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 2. Collection에 문서 임베딩 저장

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader


# 문서 로드 및 분할
loader = PyPDFLoader("data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
docs = text_splitter.split_documents(pages)

# collection에 문서 추가
collection.add(
    ids = [str(i) for i in range(len(docs))],
    documents=[i.page_content for i in docs],
    metadatas=[i.metadata for i in docs]
)

#### 3. Collection 로드

In [4]:
# name에 collection 이름, embedding_function은 collection 저장 시 지정한 임베딩 모델 지정
collection = client.get_collection(name="korean_law", embedding_function=embedding_function)
collection

Collection(name=korean_law)

### Collection 내 문서 검색

In [5]:
# query_texts는 검색할 쿼리
# n_results는 검색할 결과 개수
# where는 메타데이터 조건
# 1 페이지에서 직업 선택의 자유와 유사한 청크 3개 검색
collection.query(
    query_texts = ["직업 선택의 자유"],
    n_results = 3,
    where = {"page": 1},
)

{'ids': [['8', '4', '5']],
 'embeddings': None,
 'documents': [['관이 발부한 영장을 제시하여야 한다.\n \n제17조 모든 국민은 사생활의 비밀과 자유를 침해받지 아니한다.\n \n제18조 모든 국민은 통신의 비밀을 침해받지 아니한다.\n \n제19조 모든 국민은 양심의 자유를 가진다.\n \n제20조 ①모든 국민은 종교의 자유를 가진다.\n②국교는 인정되지 아니하며, 종교와 정치는 분리된다.',
   '법제처                                                            2                                                       국가법령정보센터\n대한민국헌법\n \n       제2장 국민의 권리와 의무\n \n제10조 모든 국민은 인간으로서의 존엄과 가치를 가지며, 행복을 추구할 권리를 가진다. 국가는 개인이 가지는 불가침\n의 기본적 인권을 확인하고 이를 보장할 의무를 진다.\n \n제11조 ①모든 국민은 법 앞에 평등하다. 누구든지 성별ㆍ종교 또는 사회적 신분에 의하여 정치적ㆍ경제적ㆍ사회적ㆍ\n문화적 생활의 모든 영역에 있어서 차별을 받지 아니한다.\n②사회적 특수계급의 제도는 인정되지 아니하며, 어떠한 형태로도 이를 창설할 수 없다.\n③훈장등의 영전은 이를 받은 자에게만 효력이 있고, 어떠한 특권도 이에 따르지 아니한다.',
   '②사회적 특수계급의 제도는 인정되지 아니하며, 어떠한 형태로도 이를 창설할 수 없다.\n③훈장등의 영전은 이를 받은 자에게만 효력이 있고, 어떠한 특권도 이에 따르지 아니한다.\n \n제12조 ①모든 국민은 신체의 자유를 가진다. 누구든지 법률에 의하지 아니하고는 체포ㆍ구속ㆍ압수ㆍ수색 또는 심문\n을 받지 아니하며, 법률과 적법한 절차에 의하지 아니하고는 처벌ㆍ보안처분 또는 강제노역을 받지 아니한다.\n②모든 국민은 고문을 받지 아니하며, 형사상 자기에게 불리한 진술을 강요당하지 아니한다

In [6]:
# $eq - 일치 (string, int, float)
# $ne - 불일치 (string, int, float)
# $gt - 초과 (int, float)
# $gte - 이상 (int, float)
# $lt - 미만 (int, float)
# $lte - 이하 (int, float)
# 5 페이지 이후의 청크 중에서 직업 선택의 자유와 관련한 문서 3개 검색
collection.query(
    query_texts = ["직업 선택의 자유"],
    n_results = 3,
    where = {"page": {"$gte": 5}}
)

{'ids': [['47', '49', '48']],
 'embeddings': None,
 'documents': [['사항은 법률로 정한다.\n \n       제9장 경제\n \n제119조 ①대한민국의 경제질서는 개인과 기업의 경제상의 자유와 창의를 존중함을 기본으로 한다.\n②국가는 균형있는 국민경제의 성장 및 안정과 적정한 소득의 분배를 유지하고, 시장의 지배와 경제력의 남용을 방\n지하며, 경제주체간의 조화를 통한 경제의 민주화를 위하여 경제에 관한 규제와 조정을 할 수 있다.\n \n제120조 ①광물 기타 중요한 지하자원ㆍ수산자원ㆍ수력과 경제상 이용할 수 있는 자연력은 법률이 정하는 바에 의하\n여 일정한 기간 그 채취ㆍ개발 또는 이용을 특허할 수 있다.\n②국토와 자원은 국가의 보호를 받으며, 국가는 그 균형있는 개발과 이용을 위하여 필요한 계획을 수립한다.\n \n제121조 ①국가는 농지에 관하여 경자유전의 원칙이 달성될 수 있도록 노력하여야 하며, 농지의 소작제도는 금지된다.\n②농업생산성의 제고와 농지의 합리적인 이용을 위하거나 불가피한 사정으로 발생하는 농지의 임대차와 위탁경영',
   '법제처                                                            13                                                       국가법령정보센터\n대한민국헌법\n⑤국가는 농ㆍ어민과 중소기업의 자조조직을 육성하여야 하며, 그 자율적 활동과 발전을 보장한다.\n \n제124조 국가는 건전한 소비행위를 계도하고 생산품의 품질향상을 촉구하기 위한 소비자보호운동을 법률이 정하는 바\n에 의하여 보장한다.\n \n제125조 국가는 대외무역을 육성하며, 이를 규제ㆍ조정할 수 있다.\n \n제126조 국방상 또는 국민경제상 긴절한 필요로 인하여 법률이 정하는 경우를 제외하고는, 사영기업을 국유 또는 공유\n로 이전하거나 그 경영을 통제 또는 관리할 수 없다.\n \n제127조 ①국가는

In [7]:
collection.query(
    query_texts = ["직업 선택의 자유"],
    n_results = 3,
    where = {"page": 1},
    where_document = {"$contains": "직업"}
)

{'ids': [['7']],
 'embeddings': None,
 'documents': [['아니라고 인정될 때 또는 정식재판에 있어서 피고인의 자백이 그에게 불리한 유일한 증거일 때에는 이를 유죄의 증\n거로 삼거나 이를 이유로 처벌할 수 없다.\n \n제13조 ①모든 국민은 행위시의 법률에 의하여 범죄를 구성하지 아니하는 행위로 소추되지 아니하며, 동일한 범죄에\n대하여 거듭 처벌받지 아니한다.\n②모든 국민은 소급입법에 의하여 참정권의 제한을 받거나 재산권을 박탈당하지 아니한다.\n③모든 국민은 자기의 행위가 아닌 친족의 행위로 인하여 불이익한 처우를 받지 아니한다.\n \n제14조 모든 국민은 거주ㆍ이전의 자유를 가진다.\n \n제15조 모든 국민은 직업선택의 자유를 가진다.\n \n제16조 모든 국민은 주거의 자유를 침해받지 아니한다. 주거에 대한 압수나 수색을 할 때에는 검사의 신청에 의하여 법\n관이 발부한 영장을 제시하여야 한다.\n \n제17조 모든 국민은 사생활의 비밀과 자유를 침해받지 아니한다.\n \n제18조 모든 국민은 통신의 비밀을 침해받지 아니한다.']],
 'uris': None,
 'data': None,
 'metadatas': [[{'creationdate': '2024-04-01T21:26:24+09:00',
    'creator': 'PyPDF',
    'moddate': '2024-04-01T21:26:24+09:00',
    'page': 1,
    'page_label': '2',
    'producer': 'iText 2.1.7 by 1T3XT',
    'source': 'data/대한민국헌법(헌법)(제00010호)(19880225).pdf',
    'total_pages': 14}]],
 'distances': [[566.5131436867309]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'doc